In [5]:
import yfinance as yf
import pandas as pd

def download_top_stocks_by_volume():
    # Download S&P 500 ticker symbols
    snp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    data = pd.read_html(snp500_url)
    sp500_tickers = data[0]['Symbol'].tolist()

    # Create an empty dictionary to store ticker symbols and their volumes
    volumes = {}

    # Defining Start & End Date
    start_date = "2010-01-01"
    end_date = "2021-01-01"

    # Iterate through each ticker symbol and get its historical data
    for ticker in sp500_tickers:
        try:
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)
            volumes[ticker] = data['Volume'].mean()  # Use mean volume for simplicity
        except Exception as e:
            print(f"Error downloading data for {ticker}: {e}")

    # Sort the dictionary by volume in descending order
    sorted_volumes = sorted(volumes.items(), key=lambda x: x[1], reverse=True)

    # Take the top 100 stocks
    top_100_tickers = [ticker[0] for ticker in sorted_volumes[:100]]

    # Download historical data for the top 100 stocks
    for ticker in top_100_tickers:
        try:
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)
            data.to_csv(f"./SnP_100/{ticker}.csv")
            # Do something with the data, e.g., save it to a file or process it further
            print(f"Downloaded data for {ticker}")
        except Exception as e:
            print(f"Error downloading data for {ticker}: {e}")


In [ ]:
download_top_stocks_by_volume()

In [8]:
snp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
data = pd.read_html(snp500_url)
sp500_tickers = data[0]['Symbol'].tolist()

volumes = {}

# Defining Start & End Date
start_date = "2010-01-01"
end_date = "2021-01-01"

# Iterate through each ticker symbol and get its historical data
for ticker in sp500_tickers:
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        volumes[ticker] = data['Volume'].mean()  # Use mean volume for simplicity
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")

# Sort the dictionary by volume in descending order
sorted_volumes = sorted(volumes.items(), key=lambda x: x[1], reverse=True)

# Take the top 100 stocks
top_100_tickers = [ticker[0] for ticker in sorted_volumes[:100]]


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2010-01-01 -> 2021-01-01)')

1 Failed download:
['CEG']: Exception("%ticker%: Data doesn't exist for startDate = 1262322000, endDate = 1609477200")

1 Failed download:
['GEHC']: Exception("%ticker%: Data doesn't exist for startDate = 1262322000, endDate = 1609477200")

1 Failed download:
['KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1262322000, endDate = 1609477200")

1 Failed download:
['VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1262322000, endDate = 1609477200")


In [9]:
top_100_tickers

['AAPL',
 'BAC',
 'TSLA',
 'AMZN',
 'GOOGL',
 'GOOG',
 'AMD',
 'T',
 'ABNB',
 'AAL',
 'AMAT',
 'MO',
 'ABT',
 'AIG',
 'ABBV',
 'BK',
 'AES',
 'AXP',
 'BAX',
 'AFL',
 'BKR',
 'BBWI',
 'APA',
 'ADM',
 'AMGN',
 'ADBE',
 'A',
 'ANET',
 'APH',
 'AEP',
 'ACN',
 'ALL',
 'MMM',
 'ADI',
 'AKAM',
 'ADSK',
 'BALL',
 'AMT',
 'ADP',
 'APTV',
 'ACGL',
 'ALK',
 'AEE',
 'AON',
 'AOS',
 'AMCR',
 'AMP',
 'APD',
 'LNT',
 'AME',
 'ALB',
 'AXON',
 'AWK',
 'ALGN',
 'AJG',
 'AVY',
 'AVB',
 'ALLE',
 'AIZ',
 'ARE',
 'ATO',
 'ANSS',
 'AZO',
 'BRK.B',
 'BF.B',
 'CSCO',
 'C',
 'CMCSA',
 'KO',
 'BSX',
 'BMY',
 'SCHW',
 'COP',
 'CCL',
 'CVX',
 'CARR',
 'BA',
 'CAT',
 'BBY',
 'CF',
 'CFG',
 'CAG',
 'CTSH',
 'BX',
 'CNP',
 'CL',
 'COF',
 'CNC',
 'CDNS',
 'CAH',
 'BWA',
 'AVGO',
 'CBRE',
 'CMS',
 'CMA',
 'CI',
 'CPB',
 'COR',
 'KMX',
 'CME']

In [1]:
def OHLCV(data, stock_name):
    data = data.drop('Adj Close', axis = 1)

    upDatedColumns = {'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}
    data.rename(columns = upDatedColumns , inplace = True)
    data['volume'] = data['volume'].astype(float)
    
    data.iloc[:, 1:] = data.iloc[:, 1:].apply(lambda x: x.astype(float))

    data['timestamp'] = pd.to_datetime(data['date'])
    data = data.reindex(columns =['timestamp', 'open','high', 'low', 'close', 'volume'])
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
    data = data.sort_values(by='timestamp' , ascending = True)
    data = data.reset_index(drop = True)

    data = data.groupby('timestamp').last().reset_index()

    data.set_index('timestamp',inplace = True)
    data.to_csv(f"./daily/{stock_name}.csv")

In [20]:
name = top_100_tickers[0]

data = pd.read_csv(f"./SnP_100/{name}.csv")
OHLCV(data, name)

In [21]:
for ticker in top_100_tickers:
    name = ticker
    data = pd.read_csv(f"./SnP_100/{ticker}.csv")
    OHLCV(data,name)
    

In [6]:
!zipline bundles

csvdir <no ingestions>
quandl 2023-12-26 11:48:49.209579
quantopian-quandl <no ingestions>
spStocks 2023-12-26 11:52:07.743594
spStocks 2023-12-26 11:49:31.893752
spStocks 2023-12-14 11:07:17.565899
spStocks 2023-12-14 08:41:15.735819
spStocks 2023-12-14 08:41:01.870597
spStocks 2023-12-14 08:40:45.601679
spStocks 2023-12-14 08:39:31.567706
spStocks 2023-12-13 07:34:23.012887
spStocks 2023-12-13 07:33:25.680991
spStocks 2023-12-13 07:29:27.845620


In [7]:
!zipline ingest -b spStocks

[2023-12-26T18:00:57+0600-INFO][zipline.data.bundles.core]
 Ingesting spStocks
Loading custom pricing data:   [####################################]  100%
Merging daily equity files:  [####################################]    


In [3]:
import pandas as pd
data = pd.read_csv('./SnP_100/AAPL.csv')
OHLCV(data, "AAPL")